In [1]:
import pandas as pd
from pathlib import Path
import cbpro
import itertools
import time 
import matplotlib.pyplot as plt
import datetime as datetime
from datetime import datetime
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import statistics
from plotly.subplots import make_subplots
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
from dash import Dash, html, dcc
import dash_leaflet as dl
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [2]:
client = cbpro.PublicClient()
client.get_product_trades(product_id='ETH-USD')

symbol = 'ETH-USD'

def trades ():
    
    client = cbpro.PublicClient()
    client.get_product_trades(product_id='ETH-USD')

    symbol = 'ETH-USD'
    
    get_trades = client.get_product_trades(product_id=f'{symbol}')
    
    df = list(itertools.islice(get_trades, 20000))
    
    df = pd.DataFrame(df[:-1])
    
    df['price'] = pd.to_numeric(df['price'])
    df['size'] = pd.to_numeric(df['size'])

    df['trade_dollar_size'] = df['price'] * df['size']
    df ['change_in_price'] = df['price'].diff()

    df ['change_in_size'] = df['size'].diff()

    df['trade_dollar_size'] = pd.to_numeric(df['price'])
    df['change_in_price'] = pd.to_numeric(df['size'])
    df['change_in_size'] = pd.to_numeric(df['price'])

    df[['price', 'size','trade_dollar_size','change_in_price']].describe()

    df.drop(['trade_id'], axis=1)
    
    df.loc[df['side'] == 'buy', 'size'] = df['size'] * -1
    df.loc[df['side'] == 'sell', 'size'] = df['size'] * 1
    
#     df.to_csv(r"D:\Trading\Bot\Docs/ETHUSD Trades.csv",index=False)
    
#     df = pd.read_csv("D:\Trading\Bot\Docs/ETHUSD Trades.csv")
    
    df['time'] = df['time'].values.astype('datetime64[s]')
    
    df_5m = df.resample('5min', on='time').agg({'price':'mean', 'size':'sum','side':'count'}).rename(columns={'price':'average price','size': 'sum of size','side':'volume'})

    df_5m['change_in_price'] = df_5m['average price'].diff()
    df_5m['change_in_size'] = df_5m['sum of size'].diff()
    df_5m['change_in_volume'] = df_5m['volume'].diff()

    df_5m.to_csv(r"D:\Trading\Bot\Docs/ETHUSD Trades_5m.csv",index=True)
    
    df_5m = pd.read_csv(r"D:\Trading\Bot\Docs/ETHUSD Trades_5m.csv")
    
    return(df_5m)

In [3]:
df_5m = trades()
df_5m

,time,average price,sum of size,volume,change_in_price,change_in_size,change_in_volume
0,2022-11-24 07:25:00,1211.873033,16.632235,1098,NaN,NaN,NaN
1,2022-11-24 07:30:00,1211.416636,-410.994664,3627,-0.456396,-427.626898,2529.0
2,2022-11-24 07:35:00,1211.330834,-1062.682625,4868,-0.085802,-651.687961,1241.0
3,2022-11-24 07:40:00,1211.529961,73.062155,1532,0.199127,1135.744780,-3336.0
4,2022-11-24 07:45:00,1211.521500,-163.597872,1953,-0.008461,-236.660027,421.0
5,2022-11-24 07:50:00,1210.583502,-566.544425,2353,-0.937998,-402.946554,400.0
6,2022-11-24 07:55:00,1207.270689,87.494853,1958,-3.312812,654.039278,-395.0
7,2022-11-24 08:00:00,1205.999036,-1.363706,1960,-1.271654,-88.858559,2.0
8,2022-11-24 08:05:00,1204.000646,12.583304,650,-1.998390,13.947010,-1310.0


In [4]:
# while True:
#     if (datetime.now().minute % 5 == 0) & (datetime.now().second == 0) :
#         df_5m = trades()
#         break
#         print(df_5m)

In [5]:
# def trades_5m():
#     df = pd.read_csv("D:\Trading\Bot\Docs/ETHUSD Trades.csv")
    
#     df['time'] = df['time'].values.astype('datetime64[s]')
    
#     df_5m = df.resample('5min', on='time').agg({'price':'mean', 'size':'sum','side':'count'}).rename(columns={'price':'average price','size': 'sum of size','side':'volume'})

#     df_5m['change_in_price'] = df_5m['average price'].diff()
#     df_5m['change_in_size'] = df_5m['sum of size'].diff()
#     df_5m['change_in_volume'] = df_5m['volume'].diff()

#     df_5m.to_csv(r"D:\Trading\Bot\Docs/ETHUSD Trades_5m.csv",index=True)
    
#     return(df_5m)

In [6]:
app = dash.Dash(__name__)

color = '#F3ECB0',

app.layout = html.Div(
   [ html.Div([
        html.H1('Live Update'),
        html.Div(id='live-update-text'),
        dcc.Graph(id='fig_v',
                  responsive = True),
        dcc.Interval(
            id='fig_1_update',
            interval=1*5000, # in milliseconds
            n_intervals=0
        )
    ]),
    
       html.Div([
        html.Div(id='live-update-text'),
        dcc.Graph(id='fig_p',
                  responsive = True),
        dcc.Interval(
            id='fig_2_update',
            interval=1*5000, # in milliseconds
            n_intervals=0
        )
    ])]
)

In [ ]:
@app.callback(Output('fig_v','figure'),
              Output('fig_p', 'figure'),
              Input('fig_1_update', 'n_intervals'))
def update_graph_live(n):
    
    df_5m = trades()
    df_5m = df_5m.round(2)
    
#     fig = px.scatter(df_5m, x='sum of size',
#                      y='average price',
#                      text='change_in_price',
#                      size='volume',color='time',
#                      template="plotly_dark",
#                      title='Size v Price')
#     fig.update_layout(autosize=True)
#     fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='Gray')
    
    fig_v = px.scatter(df_5m, x='volume',y='average price', trendline="ols",text = 'change_in_price', color='time',template="plotly_dark",size='volume',title='Volume Distribution & Average Price')
    
    fig_p = px.scatter(df_5m, x='time',
                     y='average price',
                     trendline="ols",
                     text='sum of size',
                     size='volume',
                     color='time',
                     template="plotly_dark",
                     title='Price v Change Time')
    fig_v.update_layout(plot_bgcolor='#212121',paper_bgcolor='#212121')
    fig_v.update_yaxes(showgrid=False)
    fig_v.update_xaxes(showgrid=False)
    
    
    fig_p.update_layout(plot_bgcolor='#212121',paper_bgcolor='#212121')
    fig_p.update_yaxes(showgrid=False)
    fig_p.update_xaxes(showgrid=False)
    
#     fig = go.Figure(go.Waterfall(x=df_5m['time'],
#                              y=df_5m['change_in_price'],
#                              orientation = 'v',
#                              textposition = 'outside', 
#                              text = df_5m['sum of size'],
#                              hovertext = ['x','y'],
#                              decreasing = {"marker":{"line":{"color":"red", "width":2}}},
#                              increasing = {"marker":{"color":"Green"}},
#                          ))
#     fig.update_layout(autosize=True)
    
#     fig.add_trace(go.Bar(x=df_5m['time'], y=df_5m['volume']))

#     fig.append_trace({
#         'x': df_5m['time'],
#         'y': df_5m['change_in_price'],
#         'name': 'Volume',
#         'mode': 'lines+markers',
#         'type': 'scatter'
#     }, 1, 1)

    return (fig_v, fig_p)

if __name__ == '__main__':
    app.run_server(debug=True,use_reloader=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


In [ ]:
# df = pd.read_csv("D:\Trading\Bot\Docs/ETHUSD Trades.csv")

In [ ]:
# # df['time'] = pd.to_datetime(df['time'],utc=True)
# df['time'] = df['time'].values.astype('datetime64[s]')

In [ ]:
# df_5m = df.resample('5min', on='time').agg({'price':'mean', 'size':'sum','side':'count'}).rename(columns={'price':'average price','size': 'sum of size','side':'volume'})

# df_5m['change_in_price'] = df_5m['average price'].diff()
# df_5m['change_in_size'] = df_5m['sum of size'].diff()
# df_5m['change_in_volume'] = df_5m['volume'].diff()

# df_5m.to_csv(r"D:\Trading\Bot\Docs/ETHUSD Trades_5m.csv",index=True)

In [ ]:
# df_5m = pd.read_csv('D:\Trading\Bot\Docs/ETHUSD Trades_5m.csv')

# fig = px.scatter(df_5m, x='sum of size',y='average price',size='volume',color='time', title='Average Price v Size')
# fig.update_layout(autosize=True)
# fig.show()

In [ ]:
# fig = px.scatter(df_5m, x='volume',y='average price', text = 'sum of size', color='time',size='volume',title='Volume Distribution & Average Price')
# fig.update_layout(autosize=True)
# fig.show()

In [ ]:
# df_5m = trades()

# fig = px.bar(df_5m, x='time', y='volume',color='average price',text='average price',title='Time & Volume Distribution Bars')
# fig.update_layout(autosize=True)
# fig.show()

In [ ]:
# fig = px.scatter(df_5m, x='change_in_size',y='change_in_price',text='sum of size',size='volume',color='time', title='Change in Price & Change in Price')
# fig.update_layout(autosize=True)
# fig.show()

In [ ]:
# df_5m.tail(20)

In [ ]:
# df_5m = df_5m.tail(10).round(2)

# fig = go.Figure(go.Waterfall(x=df_5m['time'],
#                              y=df_5m['change_in_price'],
#                              orientation = 'v',
#                              textposition = 'outside', 
#                              text = df_5m['sum of size'],
#                              hovertext = ['x','y'],
#                              decreasing = {"marker":{"line":{"color":"red", "width":2}}},
#                              increasing = {"marker":{"color":"Green"}},
#                          ))
# fig.update_layout(autosize=True)
# fig.show()

In [ ]:
# app = Dash(__name__)

# app.layout = html.Div(children=[
#     html.H1(children='Waterfall Graph'),

#     html.Div(children='''
#         A guide graph
#     '''),

#     dcc.Graph(
#         id='example-graph',
#         figure=fig
#     )
# ])

In [ ]:
# if __name__ == '__main__':
#     app.run_server(debug=True,use_reloader=False)